# TTP

In [47]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [48]:
%pip install openpyxl

df = pd.read_excel('../data/processed/DB People.xlsx', sheet_name='DB')

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [49]:
#Visualizar que la base de datos queda bien cargada
df.head()

,GERENCIA,Clave,Nombre,Organización,Departamento,Fecha,Hora,Terminal,Tipo,Tipo Acceso
0,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-03,07:36:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada
1,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-03,17:17:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida
2,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,05:02:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada
3,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,06:02:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida
4,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,09:58:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada


In [50]:
# ...existing code...

# Define los horarios de referencia
horarios = [pd.to_datetime('00:00').time(), pd.to_datetime('08:00').time(), pd.to_datetime('16:00').time()]

def horario_marcado(row):
    # Combina fecha y hora para obtener un datetime completo
    fecha = row['Fecha'] if 'Fecha' in row else pd.Timestamp.today().date()
    hora = row['Hora']
    # Convierte 'hora' a objeto time si es string
    if isinstance(hora, str):
        hora = pd.to_datetime(hora).time()
    dt = pd.Timestamp.combine(fecha, hora)
    tipo = row['Tipo Acceso'].lower()
    for h in horarios:
        ref = pd.Timestamp.combine(fecha, h)
        if tipo == 'entrada':
            if ref - pd.Timedelta(hours=6) <= dt <= ref + pd.Timedelta(hours=2):
                return h.strftime('%H:%M')
        elif tipo == 'salida':
            if ref - pd.Timedelta(hours=4) <= dt <= ref + pd.Timedelta(hours=4):
                return h.strftime('%H:%M')
    return np.nan

df['Horario Entrada'] = df.apply(lambda row: horario_marcado(row) if row['Tipo Acceso'].lower() == 'entrada' else np.nan, axis=1)
df['Horario Salida'] = df.apply(lambda row: horario_marcado(row) if row['Tipo Acceso'].lower() == 'salida' else np.nan, axis=1)

# Visualiza el resultado
df[['Fecha', 'Hora', 'Tipo Acceso', 'Horario Entrada', 'Horario Salida']]
# ...existing code...

,Fecha,Hora,Tipo Acceso,Horario Entrada,Horario Salida
0,2025-06-03,07:36:00,Entrada,08:00,NaN
1,2025-06-03,17:17:00,Salida,NaN,16:00
2,2025-06-04,05:02:00,Entrada,08:00,NaN
3,2025-06-04,06:02:00,Salida,NaN,08:00
4,2025-06-04,09:58:00,Entrada,08:00,NaN
...,...,...,...,...,...
4747,2025-06-11,17:18:00,Salida,NaN,16:00
4748,2025-06-12,07:40:00,Entrada,08:00,NaN
4749,2025-06-12,14:11:00,Salida,NaN,16:00
4750,2025-06-12,14:14:00,Entrada,16:00,NaN


In [51]:
#Ahora quiero junta la columna Horario Entrada y Salida
df['Horario Combinado'] = df.apply(lambda row: f"{row['Horario Entrada']} - {row['Horario Salida']}" if pd.notna(row['Horario Entrada']) and pd.notna(row['Horario Salida']) else (row['Horario Entrada'] if pd.notna(row['Horario Entrada']) else row['Horario Salida']), axis=1)
df

,GERENCIA,Clave,Nombre,Organización,Departamento,Fecha,Hora,Terminal,Tipo,Tipo Acceso,Horario Entrada,Horario Salida,Horario Combinado
0,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-03,07:36:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,08:00,NaN,08:00
1,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-03,17:17:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN,16:00,16:00
2,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,05:02:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,08:00,NaN,08:00
3,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,06:02:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN,08:00,08:00
4,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,09:58:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,08:00,NaN,08:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4747,People,38613828,MARIANA SANCHEZ QUIROGA,Bavaria & CIA S.C.A,BP SUPPLY MANAGER,2025-06-11,17:18:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN,16:00,16:00
4748,People,38613828,MARIANA SANCHEZ QUIROGA,Bavaria & CIA S.C.A,BP SUPPLY MANAGER,2025-06-12,07:40:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,08:00,NaN,08:00
4749,People,38613828,MARIANA SANCHEZ QUIROGA,Bavaria & CIA S.C.A,BP SUPPLY MANAGER,2025-06-12,14:11:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN,16:00,16:00
4750,People,38613828,MARIANA SANCHEZ QUIROGA,Bavaria & CIA S.C.A,BP SUPPLY MANAGER,2025-06-12,14:14:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,16:00,NaN,16:00


In [52]:
#Ahora quiero guardar un CSV con el df nuevo
df.to_csv('../data/processed/DB People con Horarios.csv', index=False)